<!-- ![](./texmf/logos/header.png) -->
<img src="./texmf/logos/header.png",width=100%>

# The Twinkles 1 Strong Lens Population

*Bryce Kalmbach, Phil Marshall, Curtis McCully and others*

The Twinkles 1 field is overloaded with supernovae and strongly lensed quasars. In this Note we describe the input lens population, and how the lens systems are 'sprinkled' into the PhoSim instance catalogs, having been matched by redshift and stellar mass.

## The OM10 Lensed Quasar Mock Catalog

PJM: Brief description, with links.

**JBK**: Selection of Twinkles subset.

### The Twinkles-OM10 Catalog

The lens system catalog used by Twinkles is found in the data folder of the repository and is labeled as [twinkles_lenses_v2.fits](https://github.com/LSSTDESC/Twinkles/blob/master/data/twinkles_lenses_v2.fits). We started by selecting the subset of 872 OM10 lenses that made up rung 4 in the [Strong Lens Time Delay Challenge](http://timedelaychallenge.org/index.html) ([Liao et al. 2015](https://arxiv.org/abs/1409.1254)). This set was chosen since it matches the LSST best (discussed in issue [#117](https://github.com/LSSTDESC/Twinkles/issues/117)).  **Question for PJM: What does matches LSST best mean?** However, the original OM10 catalog is incomplete and lacks sizes for the foreground lens galaxies. In addition, our image simulations require an SED for the lens galaxy of each system. Therefore, we decided to assign stellar masses to our OM10 lens galaxies based upon SL2S data and then use this information to assign an SED and radial size from an appropriate CATSIM galaxy.

#### Matching CATSIM galaxies to lensed systems

We performed the matching using code in a jupyter notebook [here](https://github.com/LSSTDESC/Twinkles/blob/master/examples/notebooks/MatchingLensGalaxies.ipynb). The process involved three main steps:

1. Use SL2S data to generate a Gaussian Mixture Model (GMM) to estimate stellar mass of lens galaxies based upon redshift and velocity dispersion.
    * We use the data from Table 3 of [Sonnenfeld et al. (2013)](https://arxiv.org/abs/1307.4759).
    * Derive the GMM using the [XDGMM](https://github.com/tholoien/XDGMM) and [empriciSN](https://github.com/tholoien/empiriciSN) packages by Tom Holoien.
2. Assign a stellar mass to the OM10 systems based upon this GMM and the OM10 redshifts and velocity dispersions.
3. Use the updated OM10 catalog with stellar masses, redshifts and ellipticities to match against CATSIM galaxies and use best fit galaxies to add SED and radius information to OM10 catalog.
    * Matched OM10 galaxies to CATSIM galaxies with a 20% error limit on the three matching parameters.
    * 11 OM10 lens galaxies did not have matches within 20% in all three parameters to a CATSIM galaxy. These were not included in the final catalog.

After the matching process we had a final catalog of 861 OM10 systems with additional size and SED information needed to generate images. This final catalog was then used as the input for `The Sprinkler`.

## `The Sprinkler` Concept and Code

**JBK**: Introduce `The Sprinkler` concept, including input and output, and any principles involved. Note: This is mostly taken straight from previous documentation [here](https://github.com/LSSTDESC/Twinkles/blob/master/doc/SLSimDocumentation/strong_lensing_review.ipynb).

`The Sprinkler` is the [code](https://github.com/LSSTDESC/Twinkles/blob/master/python/desc/twinkles/sprinkler.py) that matches OM10 lens systems to AGNs in an instance catalog and replaces the instance catalog AGN with one of our lensed systems in the final PhoSim input catalog. To do this we add a step in the CATSIM `final pass` method when creating instance catalogs for PhoSim input and manipulate the array of catalog results before they are printed to file. The first step is loading the OM10-Twinkles catalog we described in the previous section. We then go through the instance catalog AGNs one at a time and see if they match a lens system from our OM10-Twinkles catalog. The matching is done by taking the redshift and magnitude of the CATSIM AGN and selecting any available lens systems with a source AGN that has a redshift within 0.1 dex and an LSST magnorm value (magnitude in a simulated filter with one bin at 500nm) within .25 mags.

Once the possible matches to a given instance catalog AGN are found there is a chance that it will be replaced that is set by the density parameter in the sprinkler class. In the current Twinkles run it is set at 1.0 so that 198 lens systems are sprinkled into the Twinkles instance catalogs.

If the AGN is chosen to be sprinkled then we proceed with the sprinkler and if there are multiple matches from the OM10-Twinkles catalog we randomly choose one to use as we proceed. First, we remove any disc and bulge information for that galaxy from the instance catalog leaving only the AGN data and copy this AGN entry creating 2 or 4 entries in the catalog depending upon the number of images the OM10 system we are sprinkling in has in total. Next we use the OM10 data for each image to shift the location of the AGN in each image to the correct location and adjust the magnitude for each entry depending on the OM10 magnification parameter for each AGN image. At this point we also adjust the time delay for each image with OM10 info and give all the images the same source redshift from the OM10 entry. Once the new entries are ready they are appended to the instance catalog with a new instance catalog id tag that gives us the ID of the Bulge galaxy in the instance catalog to which it is associated and the ID number of the OM10-Twinkles catalog system that is used.

The next step is to take the original entry in the instance catalog and add in the information for the foreground lens galaxy. Therefore, in this step the AGN and Disc data for the original galaxy are cleared and the redshift and magnitude for the lens galaxy from the OM10 system is added into the instance catalog. Here we also add in the radius of the lens galaxy and the sed file we want to use (the information added to the original OM10 systems in the process described in the last section). We also add in the OM10 position angles and ellipticities at this point.

Once we have gone through this process for all AGN in the instance catalog then we return to the normal instance catalog processing to write the information out to file.

**JBK**: Show Sprinkler operating on OM10 sample made in previous section and producing an example (or the actual!) instance catalog.

> The matching of lensed systems seemed to take place outside `the sprinkler` - why is this? Do we want to refactor so that `the sprinkler` handles everything? I think this might be helpful...

## Results

JBK: Plots showing successful instance catalog generation: 1) Example system, showing sky positions of lensed quasar images and lens galaxy, compared to standard OM10 representation. 2) Distributions of lens galaxy and source quasar magnitude, to show that matching works well. etc.

## DIscussion and Conclusions

JBK and PJM to iterate on:

`sprinkler` seems to work well, inserting lensed quasars into PhoSim inputs. Speed of operation? Is this the rate-limiting step? 

Scaling up to larger sky areas - any problems foreseen?

Quasar host galaxies are ignored for now, but could be matched and inserted (Li and Wiesner in prep).

Lensed supernovae? OM10 produced a mock SN catalog that could be used. 

# Appendix: LSST DESC Notes `ipynb` Reference

You can delete all of this whenever you're ready.

## Tables

Tables can be fiddly in `Markdown`. A good place to start is an online table generator like [this one](http://www.tablesgenerator.com/markdown_tables). Then, you'll need some patience. For more on table formatting, see the [`Markdown` cheat-sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables).

|   A   |   B   |      C         |  D  |
|:-----:|:-----:|:--------------:|:---:|
| (deg) | (kpc) | ($M_{\odot}$)  |     |
|  0.4  |  3.4  |  $10^{12.2}$   | R,S |
|  9.6  |  8.2  |  $10^{10.4}$   |  S  |


## References

You can cite papers (or anything else) by providing hyperlinks. For
example, you might have been impressed by the DESC White Paper [(LSST
Dark Energy Science Collaboration
2012)](http://arxiv.org/abs/1211.0310). It should be possible to convert
these links to latex citations automatically later.

[Liao K. et al., 2015, ApJ, 800, 26](https://arxiv.org/abs/1409.1254)
[Sonnenfeld A. et al., 2013, ApJ, 777, 98](https://arxiv.org/abs/1307.4759)

## Further Resources

For a guide to writing `Markdown` documents, check out this [useful little cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).